In [1]:
# Update dependencies to resolve import errors
!pip install diffusers transformers torch huggingface_hub safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 30.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4


In [2]:
!pip install kagglehub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 1.9 MB/s eta 0:00:00


In [3]:
import kagglehub
import os
import shutil

path = kagglehub.dataset_download("hoangtran223/itay-dataset")
print("Path to dataset files:", path)

!git clone https://github.com/bmaltais/kohya_ss.git /kaggle/working/kohya_ss
%cd /kaggle/working/kohya_ss/

!git checkout 2853f4cec850557e83882a4072cfe416cb9c57d7
!/kaggle/working/kohya_ss/setup.sh -n
print("Kohya_SS is done setting up!")

Path to dataset files: /kaggle/input/itay-dataset
Cloning into '/kaggle/working/kohya_ss'...
remote: Enumerating objects: 14955, done.
remote: Counting objects: 100% (574/574), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 14955 (delta 505), reused 440 (delta 425), pack-reused 14381 (from 4)
Receiving objects: 100% (14955/14955), 23.26 MiB | 32.85 MiB/s, done.
Resolving deltas: 100% (10493/10493), done.
/kaggle/working/kohya_ss
Note: switching to '2853f4cec850557e83882a4072cfe416cb9c57d7'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config va

In [4]:
prompt_trigger = "wny"
gender = "man"

os.makedirs("/kaggle/working/training-data/img", exist_ok=True)
os.makedirs("/kaggle/working/training-data/reg", exist_ok=True)
os.makedirs("/kaggle/working/my-dreambooth-lora", exist_ok=True)
os.makedirs("/kaggle/working/training-data/log", exist_ok=True)

!rm -r /kaggle/working/training-data/*

shutil.copytree(f"{path}", f"/kaggle/working/training-data/img/25_{prompt_trigger} {gender}")
shutil.copytree(f"/kaggle/input/regularization-images-{gender}", f"/kaggle/working/training-data/reg/1_{gender}")

'/kaggle/working/training-data/reg/1_man'

In [5]:
# !rm -rf /kaggle/working/

In [6]:
%cd /kaggle/working/kohya_ss/

num_images = len(os.listdir(f"/kaggle/working/training-data/img/25_{prompt_trigger} {gender}"))
max_train_steps = num_images * 50

# --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \

!accelerate launch --num_cpu_threads_per_process=2 "./sdxl_train_network.py" \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --train_data_dir="/kaggle/working/training-data/img" \
  --reg_data_dir="/kaggle/working/training-data/reg" \
  --output_dir="/kaggle/working/my-dreambooth-lora" \
  --logging_dir="/kaggle/working/training-data/log" \
  --resolution="1024,1024" \
  --network_alpha="16" \
  --network_dim=64 \
  --save_model_as=safetensors \
  --network_module=networks.lora \
  --text_encoder_lr=0.0004 \
  --unet_lr=0.0004 \
  --output_name="sdxl-turbo-dreambooth-lora" \
  --lr_scheduler_num_cycles="8" \
  --no_half_vae \
  --learning_rate="0.0004" \
  --lr_scheduler="constant_with_warmup" \
  --lr_warmup_steps=100 \
  --train_batch_size="1" \
  --gradient_accumulation_steps=4 \
  --max_train_steps={max_train_steps} \
  --save_every_n_epochs="1" \
  --mixed_precision="fp16" \
  --save_precision="fp16" \
  --optimizer_type="Adafactor" \
  --optimizer_args scale_parameter=False relative_step=False warmup_init=False \
  --max_data_loader_n_workers="0" \
  --gradient_checkpointing \
  --xformers \
  --enable_bucket \
  --bucket_reso_steps=32 \
  --bucket_no_upscale \
  --noise_offset=0.0 \
  --lowram \
  --mem_eff_attn

/kaggle/working/kohya_ss
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
prepare tokenizers
prepare tokenizers
Using DreamBooth method.
Using DreamBooth method.
prepare images.
found directory /kaggle/working/training-data/img/25_wny man contains 24 image files
No caption file found for 24 images.

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, AutoencoderKL
from safetensors.torch import load_file
import os

output_dir = "/kaggle/working/generated-images"
os.makedirs(output_dir, exist_ok=True)

base_model = "stabilityai/stable-diffusion-xl-base-1.0"
lora_weights = "/kaggle/working/my-dreambooth-lora/sdxl-dreambooth-lora-000005.safetensors"

vae = AutoencoderKL.from_pretrained(
    base_model,
    subfolder="vae",
    torch_dtype=torch.float16,
    use_safetensors=True
)

pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model,
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True
)

device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipe.to(device)
state_dict = load_file(lora_weights)

unet_lora_state = {k: v for k, v in state_dict.items() if "unet" in k.lower()}
text_encoder_lora_state = {k: v for k, v in state_dict.items() if "text_encoder" in k.lower()}

if unet_lora_state:
    pipe.unet.load_state_dict(unet_lora_state, strict=False)
else:
    print("No UNet LoRA weights found in the state dictionary.")

if text_encoder_lora_state:
    adjusted_text_encoder_state = {k.replace("text_encoder.", ""): v for k, v in text_encoder_lora_state.items()}
    pipe.text_encoder.load_state_dict(adjusted_text_encoder_state, strict=False)
else:
    print("No text encoder LoRA weights found in the state dictionary.")

prompt = "analog style a wny man old money aesthetic, highly detailed,  high-definition, sharp focus, intricate details, professionally retouched, 8k high definition, photorealistic, detailed face,  insanely detailed hair."
negative_prompt = "multiple heads, 2 heads, elangaled body, double image, 2 faces, multiple people, double head, nsfw, nude, porn, naked,  frilly, frilled, lacy, ruffled, victorian, (deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
num_inference_steps = 50
guidance_scale = 7.5
width = 1024
height = 1024

for i in range(5):
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        width=width,
        height=height
    ).images[0]
    
    output_path = os.path.join(output_dir, f"generated_image_{i}.png")
    image.save(output_path)
    print(f"Image saved to {output_path}")

del pipe
torch.cuda.empty_cache()